In [1]:
import sys
import os.path
sys.path.append(os.path.join('..', 'util'))

import set_compiler
set_compiler.install()

import pyximport
pyximport.install()

import numpy as np
import matplotlib.pyplot as plt

import filtering
from timer import Timer
import threading as th
import logging

%matplotlib inline

Compiling with /usr/local/bin/clang-omp


In [2]:
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.basicConfig(filename='log.txt',
                    level=logging.DEBUG,
                    format='[%(levelname)s] (%(threadName)-10s) %(message)s',
                    )

In [3]:
input_image = np.load('image.npz')['image'].astype(np.float32)

#plt.gray()

#plt.imshow(input_image)
#plt.title('original image')

#plt.figure()
#plt.imshow(input_image[1200:1800, 3000:3500])
#plt.title('before - zoom')
input_image.shape

(2404, 9184)

In [4]:
input_image

array([[ 160.25230408,  159.25239563,  155.25280762, ...,  152.10049438,
         150.10069275,  149.10079956],
       [ 159.25239563,  159.25239563,  156.25270081, ...,  153.10040283,
         150.10069275,  150.10069275],
       [ 156.25270081,  157.25259399,  155.25280762, ...,  150.10069275,
         147.10099792,  148.10090637],
       ..., 
       [ 146.65859985,  148.65840149,  153.65789795, ...,   20.04739952,
          22.04719925,   25.0468998 ],
       [ 148.65840149,  148.65840149,  151.65809631, ...,   20.04739952,
          22.04719925,   24.04700089],
       [ 148.65840149,  149.65829468,  150.65820312, ...,   21.04730034,
          22.04719925,   23.04710007]], dtype=float32)

In [88]:
# oct 31 6:20pm

# the problem you're having is because you're slicing so early that the actual median function
# isn't getting the full array to work with. that's why you're getting medians from the 3rd
# row in your first row (the first row shouldn't even be able to see the 3rd row, as it only
# takes the one below it - row2 - and the (nonexistent) one above it, which ends up just being
# a copy of itself). 

# you need to figure out a way to let the cython median function act on the full array, but still
# have each thread only concerned with determining median values for every nth row.

def per_thread_median(A,B,this_thr,evs,I):
    ''' Runs Cython median_3x3() with thread-private offset image slice
    
        Slices start at index start=0...num_threads of tmpA, and are passed in as A.
        
        Each slice waits until the previous iterations of threads controlling
        the thread_num-1, thread_num, and thread_num+1 slices have finished. 
        
        "Finished" is signaled by the Event variable in evs[] for that particular slice on 
        the previous iteration (i-1).
        
        Example: 
            For current iteration i, with thread_index n, where 0 > n > num_threads-1:
            We wait to begin median_3x3 until:
                evs[ n-1, i-1 ] is set to True
                evs[ n  , i-1 ] is set to True
                evs[ n+1, i-1 ] is set to True
        
        Nothing is returned as A is a view on tmpA in the parent function, so modifications
        happen directly to tmpA (which is a copy of the image we're filtering).
    '''
    for i in range(I):
        logger.debug('Round {}'.format(i))
        
        '''If this_thr is index 0 or max-thread, wrap-around prev/next thread idx
            Eg. this_thr == 0 --> prev_thread = max-thread idx
        
            NOTE: If only 2 threads, this should still be fine, but two out of the three checks
                  will be for the same Event. 
            Eg. For 2 threads, for thread 0, next_thr = 1, prev_thr = 1, so both will wait for 
                same Event to become true.
        '''    
        prev_thr = this_thr-1 if this_thr-1 >= 0 else evs.shape[0]-1
        next_thr = this_thr+1 if this_thr+1 < evs.shape[0] else 0
        
        if i: # for i==0 no checking is necessary
            
            logger.debug('THREAD {} waiting for Events i-1={} to be ready'.format(this_thr,i-1))

            # wait until all nearby threads have finished iteration i-1
            
            evs[ prev_thr, i-1 ].wait()  
            evs[ this_thr, i-1 ].wait() 
            evs[ next_thr, i-1 ].wait() 

        logger.debug('THREAD {} ready to proceed, i={}\n'.format(this_thr,i))

        filtering.median_3x3(A, B, 0, 1)

        logger.debug('Results: THREAD {}, i={}'.format(this_thr,i))
        logger.debug(B)
        logger.debug('\n\n')
        # set Event == True for current thread, current iteration 
        evs[this_thr,i].set()
        
        # lockstep with other threads (don't proceed until all other threads are done for i)
        for ix in range(evs.shape[0]):
            evs[ix,i].wait()
        # swap direction of filtering
        #A, B = B, A
        
        logger.debug('THREAD {} complete and Event.set(), i={}'.format(this_thr,i))
            
def py_median_3x3(image, iterations=10, num_threads=1):
    ''' repeatedly filter with a 3x3 median '''
    tmpA = image.copy()
    tmpA = tmpA[0:4,0:4].copy()
    tmpB = np.empty_like(tmpA)

    # initialize Events array, shape: [num_threads x iterations]
    events = np.empty( (num_threads, iterations), dtype=th._Event )
    
    # populate Events array
    for i in xrange(num_threads):
        events[i,:] = [th.Event() for _ in xrange(iterations)]   
    
    # initialize threads array, shape: [num_threads]
    threads = np.empty(num_threads,dtype=th.Thread)

    # populate threads array
    # each thread starts per_thread_median() with args: slice tmpA, slice tmpB, thread id, events array, num iterations
    for start in range(num_threads):
        threads[start] = th.Thread(name='t{}'.format(start), 
                                   target=per_thread_median, 
                                   args=(tmpA[start::num_threads,:],
                                         tmpB[start::num_threads,:],
                                         start,
                                         events,
                                         iterations
                                        )
                                  )
        threads[start].start()
    
    # reunite threads into master, use .join() to make sure all threads have completed
    for ix in range(num_threads):
        threads[ix].join()
        
    logger.debug('END FUNCTION\n\n')
    return tmpB

In [89]:
def numpy_median(image, iterations=10):
    ''' filter using numpy '''
    for i in range(iterations):
        padded = np.pad(image, 1, mode='edge')
        stacked = np.dstack((padded[:-2,  :-2], padded[:-2,  1:-1], padded[:-2,  2:],
                             padded[1:-1, :-2], padded[1:-1, 1:-1], padded[1:-1, 2:],
                             padded[2:,   :-2], padded[2:,   1:-1], padded[2:,   2:]))
        image = np.median(stacked, axis=2)

    return image

In [90]:
from_cython = py_median_3x3(input_image, 1, 2)


DEBUG:root:Round 0
DEBUG:root:Round 0
DEBUG:root:THREAD 0 ready to proceed, i=0

DEBUG:root:THREAD 1 ready to proceed, i=0

DEBUG:root:Results: THREAD 0, i=0
DEBUG:root:Results: THREAD 1, i=0
DEBUG:root:[[ 159.25239563  157.25259399  155.89370728  155.89370728]
 [ 157.25259399  156.25270081  157.25259399  155.89370728]]
DEBUG:root:[[ 159.25239563  156.25270081  156.89360046  156.89360046]
 [ 156.25270081  156.25270081  156.25270081  156.89360046]]
DEBUG:root:


DEBUG:root:


DEBUG:root:THREAD 1 complete and Event.set(), i=0
DEBUG:root:THREAD 0 complete and Event.set(), i=0
DEBUG:root:END FUNCTION




In [86]:
from_numpy = numpy_median(input_image[0:4,0:4], 1)


In [87]:
print input_image[0:4,0:4]
print
print "cython"
print from_cython[0,:]
print
print "numpy"
print from_numpy[0,:]
print np.where(from_cython[0,:] == from_numpy[0,:])[0].shape[0]
print from_cython[0,:].shape[0]
np.where(from_cython[0,:] == from_numpy[0,:])[0].shape[0]/float(from_cython[0,:].shape[0])

[[ 160.25230408  159.25239563  155.25280762  155.89370728]
 [ 159.25239563  159.25239563  156.25270081  156.89360046]
 [ 156.25270081  157.25259399  155.25280762  157.25259399]
 [ 154.25289917  156.25270081  155.25280762  157.25259399]]

cython
[ 159.25239563  157.25259399  155.89370728  155.89370728]

numpy
[ 159.25239563  159.25239563  156.25270081  155.89370728]
2
4


0.5

In [71]:
from_cython

array([[ 159.25239563,  157.25259399,  155.89370728,  155.89370728],
       [ 159.25239563,  156.25270081,  156.89360046,  156.89360046],
       [ 157.25259399,  156.25270081,  157.25259399,  155.89370728],
       [ 156.25270081,  156.25270081,  156.25270081,  156.89360046]], dtype=float32)

In [73]:
from_numpy

array([[ 159.25239563,  159.25239563,  156.25270081,  155.89370728],
       [ 159.25239563,  157.25259399,  156.89360046,  156.25270081],
       [ 156.25270081,  156.25270081,  156.89360046,  156.89360046],
       [ 156.25270081,  155.25280762,  156.25270081,  157.25259399]], dtype=float32)

In [30]:
assert np.all(from_cython == from_numpy)
print "CYTHON == NUMPY"

CYTHON == NUMPY


In [ ]:


with Timer() as t:
    new_image = py_median_3x3(input_image, 10, 8)

pylab.figure()
pylab.imshow(new_image[1200:1800, 3000:3500])
pylab.title('after - zoom')

print("{} seconds for 10 filter passes.".format(t.interval))
pylab.show()